In [1]:
import pandas as pd
import scipy as sci
import numpy as np
import matplotlib.pyplot as plt

In [2]:
eegF1_df = pd.read_json('file:eegData_F1.json')
eegM1_df = pd.read_json('file:eegData_M1.json')
eventF1_df = pd.read_json('file:EventJson_F1.json')
eventM1_df = pd.read_json('file:EventJson_M1.json')
accelF1_df = pd.read_json('file:accelerometerJson_F1.json')
accelM1_df = pd.read_json('file:accelerometerJson_M1.json')

In [3]:
def process_eeg_dataframe(df):
    df_raw_time_vector = df[df['electrode'] == 0]['timestamp'].to_numpy()
    time_spacing = []
    
    for i in np.arange(1,len(df_raw_time_vector)):
        time_spacing.append(df_raw_time_vector[i]-df_raw_time_vector[i-1])

        if(i>1):
            if(not time_spacing[i-1] == time_spacing[i-2]):
                print(time_spacing[i-1],time_spacing[i-2])
                
    T = df_raw_time_vector[-1] - df_raw_time_vector[0]
    T = T / np.timedelta64(1, 's')
    print('length: ',T,' seconds')
    dt = time_spacing[0]
    dt = dt / np.timedelta64(1, 's')
    print('sample spacing: ',dt,' seconds')

    time_vector = np.arange(0,T,dt)
    
    channel_list = []
    eeg_channels = []
    for channel in np.arange(5):
        channel_list.append(df[df['electrode'] == channel])
        eeg_temp = channel_list[channel]['samples'].tolist()
        eeg = []
        for a in eeg_temp:
            for b in a:
                eeg.append(b)
        eeg_channels.append(np.array(eeg))
        print('shape of channel ',channel,': ',eeg_channels[channel].shape)
    eeg_channels = np.array(eeg_channels).squeeze()
    return time_vector,eeg_channels,T

In [4]:
F1_time_vector,F1_eeg,F1_T = process_eeg_dataframe(eegF1_df)
F2_time_vector,F2_eeg,M1_T = process_eeg_dataframe(eegF1_df)

length:  296.390625  seconds
sample spacing:  0.046875  seconds
shape of channel  0 :  (75888,)
shape of channel  1 :  (75888,)
shape of channel  2 :  (75888,)
shape of channel  3 :  (75888,)
shape of channel  4 :  (75888,)
length:  296.390625  seconds
sample spacing:  0.046875  seconds
shape of channel  0 :  (75888,)
shape of channel  1 :  (75888,)
shape of channel  2 :  (75888,)
shape of channel  3 :  (75888,)
shape of channel  4 :  (75888,)


In [5]:
def process_event_dataframe(df,start_time):
    data = df.to_numpy()
    for i in np.arange(len(data)):
        data[i,0] = (data[i,0] - start_time)
    return data

In [6]:
def first_timestamp(df):
    timestamp = df['timestamp'][0]
    return timestamp

In [10]:
print(first_timestamp(eegF1_df))
print(first_timestamp(eventF1_df))
print(first_timestamp(eventF1_df)-first_timestamp(eegF1_df))

2019-07-25 22:40:11.708375
2019-07-25 22:50:22.979000
0 days 00:10:11.270625


In [12]:
print(process_event_dataframe(eventF1_df,first_timestamp(eegF1_df)))

[[Timedelta('0 days 00:10:11.270625') 12]
 [Timedelta('0 days 00:10:22.520625') 12]
 [Timedelta('0 days 00:10:33.813625') 9]
 [Timedelta('0 days 00:10:45.064625') 9]
 [Timedelta('0 days 00:10:56.313625') 4]
 [Timedelta('0 days 00:11:07.610625') 4]
 [Timedelta('0 days 00:11:18.859625') 16]
 [Timedelta('0 days 00:11:30.138625') 16]
 [Timedelta('0 days 00:11:41.391625') 26]
 [Timedelta('0 days 00:11:52.641625') 26]
 [Timedelta('0 days 00:12:03.934625') 42]
 [Timedelta('0 days 00:12:15.186625') 42]
 [Timedelta('0 days 00:12:26.481625') 8]]
